In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_logistic=pd.read_csv('./contraceptive_logistics_data.csv')
data_logistic.head()

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,75,0,21,-54,0,18,0,100.0
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3,0,3,0,0,2,0,10.0
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,0,75,22,0,53,9,0,0.0
3,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27137,2,0,0,0,2,0,0,0.0
4,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27138,45,0,2,0,43,1,0,0.0


In [5]:
data_logistic["date"]=[str(annee)+'-'+str(mois)+'-01' for mois, annee in zip(data_logistic['month'],data_logistic['year'])]
data_logistic['date']=pd.to_datetime(data_logistic['date'])
data_logistic['product_site']=[product+site for product,site in zip (data_logistic['site_code'],data_logistic['product_code'])]
b=data_logistic.groupby('product_site').agg({'product_code':'count'})
b.reset_index(inplace=True)


In [6]:
b['zeros']='N'

for index, row  in b.iterrows():
    s=data_logistic[ data_logistic['product_site']==row['product_site']]['stock_distributed'].sum()
    if(s==0):
        b.loc[index,'zeros']="O"

In [8]:
b.groupby('zeros').agg({'product_site':'count'}) #we have 385 site with any value for stock distributed. May be we need to focus on 972 for our forecasting


,product_site
zeros,
N,972
O,385


In [9]:
c=b[(b['zeros']=='N')]
c=c[c['product_code']>11]#Let's focus on the "N", particulary for those whose series have more than eleven values or periods

In [ ]:
# Utilisation d'un modèle de réseau de neurone
#!pip install tensorflow
import tensorflow as tf
import numpy as np

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [ ]:
window_size = 2
batch_size = 5
shuffle_buffer_size = len(series)
print(len(series))

In [ ]:
dataset = windowed_dataset(series, window_size, batch_size, shuffle_buffer_size)
print(dataset)
l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])


model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset,epochs=100,verbose=1)

print("Layer weights {}".format(l0.get_weights()))

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_shape=[window_size], activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.SGD(lr=8e-6, momentum=0.9)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(dataset, epochs=500, verbose=0)

In [ ]:
forecast = []

for t in range(len(series) - window_size):
 
  forecast.append(model.predict(series[t:t + window_size][np.newaxis]))
  
results = np.array(forecast)[:,0,0]

plt.figure(figsize=(10, 6))

plt.plot(time, series)
plt.plot(time[2:], results)

In [ ]:
tf.keras.metrics.mean_absolute_error(series[:31], results).numpy()